## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sklearn as skl
import pandas as pd
import tensorflow as tf
import pandas as pd 

In [2]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


Target data will be data in column 'IS_SUCCESSFUL'

Feature data will be all other columns except 'EIN' and 'NAME'

In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'
application_df.drop(columns=['EIN', 'NAME'], inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# value counts look sparse after T10=528 so I will make this the cutoff value and bucket all other types into "Other"
cutoff_value = 500

# Take value_counts series from application_df['APPLICATION_TYPE']
# Apply boolean condition to filer less than cutoff value
# Take the index from the series as the 'application_types_to_replace'
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < cutoff_value].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# value counts look sparse after C2100=1883 so I will make this the cutoff value and bucket all other types into "Other"
cutoff_value = 1800

# Take value_counts series from application_df['CLASSIFICATION']
# Apply boolean condition to filer less than cutoff value
# Take the index from the series as the 'classifications_to_replace'
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < cutoff_value].index


# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies` 
# While INCOME_AMT looks like it is numerical, what it actually is is a set of discrete income ranges which makes it categorical
categorical_columns = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']

# Encode these columns as dummies. **NEED TO SPECIFY dtype=int**
dummy_columns_df = pd.get_dummies(application_df[categorical_columns], dtype=int)

# concat the original dataframe and the dummy_columns_df. We then can drop the original categorical columns which results in us essentially replacing them with the encoded ones.
application_df_encoded = pd.concat([application_df,dummy_columns_df], axis=1)
application_df_encoded.drop(columns=categorical_columns,inplace=True)
application_df_encoded.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [11]:
# Split our preprocessed data into our features and target arrays
X = application_df_encoded.drop('IS_SUCCESSFUL', axis=1)
y = application_df_encoded['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=95, stratify=y)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [13]:
# Double check how many input dimensions we have by counting the columns
print(X_train_scaled.shape[1])

43


### Optimization Method 1
    - Adding additonal neurons based on # of inputs * 3 to the next power of 2
    - Second layer neurons will follow that power of 2 rule and be half that
    

In [14]:
next_power_of_2 = 2 ** (((43*3) - 1).bit_length())
next_power_of_2

256

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=256, activation="relu", input_dim=43))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=128, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               11264     
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 44289 (173.00 KB)
Trainable params: 44289 (173.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

### Define callback function to save every 5 Epochs

In [17]:
# Import additional dependencies
from pathlib import Path

# (1) This callback function is designed to save a checkpoint at every n epochs and it will make each checkpoint have its own sub directory
# function will also always save the last epoch no matter what
class MyCallback(tf.keras.callbacks.Callback):
    def __init__(self, base_dir, save_freq):
        super(MyCallback, self).__init__()
        self.base_dir = Path(base_dir)  # base directory for checkpoints
        self.save_freq = save_freq  # how frequent to save weights
        
        # Create the base directory if it doesn't exist
        self.base_dir.mkdir(parents=True, exist_ok=True)
        
    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_freq == 0:
            self.save_checkpoint(epoch, final=False)
    
    def on_train_end(self, logs=None):
        # Save a final checkpoint at the end of training
        last_epoch = self.params['epochs'] - 1
        self.save_checkpoint(last_epoch, final=True)
        
    def save_checkpoint(self, epoch, final=False):
        # Create a subdirectory path for the current epoch
        subdir_name = f"epoch-{epoch + 1:04d}" + ("-final" if final else "")
        epoch_dir = self.base_dir / subdir_name
        epoch_dir.mkdir(parents=True, exist_ok=True)
        
        # Define the full path for the current epoch's checkpoint
        checkpoint_filename = f"cp-{epoch + 1:04d}" + ("-final" if final else "") + ".ckpt"
        checkpoint_path = epoch_dir / checkpoint_filename
        
        # Save the model's weights
        self.model.save_weights(str(checkpoint_path))
        print(f"\nSaved weights at epoch {epoch + 1} to {checkpoint_path}")


In [18]:
# Define directory for files to save at
checkpoint_dir = Path("model 1 checkpoints")

# create callback function from class
my_callback = MyCallback(base_dir=str(checkpoint_dir), save_freq=5)

In [19]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[my_callback])

Epoch 1/100


804/804 [==============================] - 3s 2ms/step - loss: 0.5699 - accuracy: 0.7244
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5545 - accuracy: 0.7314
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5513 - accuracy: 0.7331
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5486 - accuracy: 0.7347
Epoch 5/100
791/804 [============================>.] - ETA: 0s - loss: 0.5481 - accuracy: 0.7340
Saved weights at epoch 5 to model 1 checkpoints\epoch-0005\cp-0005.ckpt
804/804 [==============================] - 2s 3ms/step - loss: 0.5482 - accuracy: 0.7342
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5465 - accuracy: 0.7345
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5460 - accuracy: 0.7344
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5451 - accuracy: 0.7362
Epoch 9/100
804/804 [============

804/804 [==============================] - 2s 3ms/step - loss: 0.5344 - accuracy: 0.7407
Epoch 60/100
801/804 [============================>.] - ETA: 0s - loss: 0.5338 - accuracy: 0.7410
Saved weights at epoch 60 to model 1 checkpoints\epoch-0060\cp-0060.ckpt
804/804 [==============================] - 2s 3ms/step - loss: 0.5343 - accuracy: 0.7407
Epoch 61/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5342 - accuracy: 0.7413
Epoch 62/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5342 - accuracy: 0.7409
Epoch 63/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5339 - accuracy: 0.7407
Epoch 64/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5340 - accuracy: 0.7406
Epoch 65/100
789/804 [============================>.] - ETA: 0s - loss: 0.5342 - accuracy: 0.7412
Saved weights at epoch 65 to model 1 checkpoints\epoch-0065\cp-0065.ckpt
804/804 [==============================] - 2s 3ms/step - loss: 0.5338 - 

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5772 - accuracy: 0.7238 - 634ms/epoch - 2ms/step
Loss: 0.577208399772644, Accuracy: 0.7238484025001526


### Optimization Attempt 2
    - Adding additional layer following power of 2 rule
  

In [39]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=256, activation="relu", input_dim=43))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=128, activation="relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=64, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 256)               11264     
                                                                 
 dense_19 (Dense)            (None, 128)               32896     
                                                                 
 dense_20 (Dense)            (None, 64)                8256      
                                                                 
 dense_21 (Dense)            (None, 1)                 65        
                                                                 
Total params: 52481 (205.00 KB)
Trainable params: 52481 (205.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [40]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [41]:
# Define directory for files to save at
checkpoint_dir2 = Path("model 2 checkpoints")

# create callback function from class
my_callback2 = MyCallback(base_dir=str(checkpoint_dir2), save_freq=5)

In [43]:
fit_model2 = nn2.fit(X_train_scaled, y_train, epochs=100, callbacks=[my_callback2])

Epoch 1/100
856/856 [==============================] - 3s 3ms/step - loss: 0.5692 - accuracy: 0.7201
Epoch 2/100
856/856 [==============================] - 2s 3ms/step - loss: 0.5567 - accuracy: 0.7283
Epoch 3/100
856/856 [==============================] - 2s 3ms/step - loss: 0.5540 - accuracy: 0.7284
Epoch 4/100
856/856 [==============================] - 2s 3ms/step - loss: 0.5528 - accuracy: 0.7288
Epoch 5/100
848/856 [============================>.] - ETA: 0s - loss: 0.5516 - accuracy: 0.7286
Saved weights at epoch 5 to model 3 checkpoints\epoch-0005\cp-0005.ckpt
856/856 [==============================] - 2s 3ms/step - loss: 0.5514 - accuracy: 0.7288
Epoch 6/100
856/856 [==============================] - 2s 3ms/step - loss: 0.5503 - accuracy: 0.7310
Epoch 7/100
856/856 [==============================] - 2s 3ms/step - loss: 0.5500 - accuracy: 0.7308
Epoch 8/100
856/856 [==============================] - 2s 3ms/step - loss: 0.5484 - accuracy: 0.7304
Epoch 9/100
856/856 [==============

856/856 [==============================] - 2s 3ms/step - loss: 0.5358 - accuracy: 0.7371
Epoch 63/100
856/856 [==============================] - 2s 3ms/step - loss: 0.5353 - accuracy: 0.7377
Epoch 64/100
856/856 [==============================] - 2s 3ms/step - loss: 0.5418 - accuracy: 0.7361
Epoch 65/100
843/856 [============================>.] - ETA: 0s - loss: 0.5354 - accuracy: 0.7364
Saved weights at epoch 65 to model 3 checkpoints\epoch-0065\cp-0065.ckpt
856/856 [==============================] - 2s 3ms/step - loss: 0.5353 - accuracy: 0.7368
Epoch 66/100
856/856 [==============================] - 2s 3ms/step - loss: 0.5349 - accuracy: 0.7372
Epoch 67/100
856/856 [==============================] - 2s 3ms/step - loss: 0.5341 - accuracy: 0.7376
Epoch 68/100
856/856 [==============================] - 2s 3ms/step - loss: 0.5350 - accuracy: 0.7372
Epoch 69/100
856/856 [==============================] - 2s 3ms/step - loss: 0.5345 - accuracy: 0.7370
Epoch 70/100
842/856 [=================

In [35]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6028 - accuracy: 0.7249 - 656ms/epoch - 2ms/step
Loss: 0.6027791500091553, Accuracy: 0.7248979806900024


### Optimization Attempt 3
    - Using PCA to reduce feature count
    - change the neural network layers based on our new feature counts

In [86]:
from sklearn.decomposition import PCA

In [87]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=32)

In [88]:
X_train_scaled_pca = pca.fit_transform(X_train_scaled)
X_test_scaled_pca = pca.fit_transform(X_test_scaled)

In [89]:
# Check how much variance is accounted for
pca.explained_variance_ratio_.sum()

0.9807530378470188

#### Now using 32 input components, will change layers accordingly

In [90]:
next_power_of_2 = 2 ** (((32*3) - 1).bit_length())
next_power_of_2

128

In [96]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=128, activation="relu", input_dim=32))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=64, activation="relu"))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=32, activation='tanh'))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_34 (Dense)            (None, 128)               4224      
                                                                 
 dense_35 (Dense)            (None, 64)                8256      
                                                                 
 dense_36 (Dense)            (None, 32)                2080      
                                                                 
 dense_37 (Dense)            (None, 1)                 33        
                                                                 
Total params: 14593 (57.00 KB)
Trainable params: 14593 (57.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [97]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [98]:
# Define directory for files to save at
checkpoint_dir3 = Path("model 3 checkpoints")

# create callback function from class
my_callback3 = MyCallback(base_dir=str(checkpoint_dir3), save_freq=5)

In [99]:
fit_model3 = nn3.fit(X_train_scaled_pca, y_train, epochs=100, callbacks=[my_callback])

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5666 - accuracy: 0.7209
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5543 - accuracy: 0.7300
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5519 - accuracy: 0.7320
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5503 - accuracy: 0.7331
Epoch 5/100
791/804 [============================>.] - ETA: 0s - loss: 0.5483 - accuracy: 0.7347
Saved weights at epoch 5 to model 1 checkpoints\epoch-0005\cp-0005.ckpt
804/804 [==============================] - 2s 3ms/step - loss: 0.5484 - accuracy: 0.7344
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5474 - accuracy: 0.7350
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5469 - accuracy: 0.7348
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5456 - accuracy: 0.7358
Epoch 9/100
804/804 [==============

804/804 [==============================] - 2s 3ms/step - loss: 0.5354 - accuracy: 0.7406
Epoch 63/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5357 - accuracy: 0.7390
Epoch 64/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5350 - accuracy: 0.7402
Epoch 65/100
795/804 [============================>.] - ETA: 0s - loss: 0.5353 - accuracy: 0.7399
Saved weights at epoch 65 to model 1 checkpoints\epoch-0065\cp-0065.ckpt
804/804 [==============================] - 2s 3ms/step - loss: 0.5348 - accuracy: 0.7403
Epoch 66/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5347 - accuracy: 0.7399
Epoch 67/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5348 - accuracy: 0.7413
Epoch 68/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5343 - accuracy: 0.7404
Epoch 69/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5345 - accuracy: 0.7408
Epoch 70/100
787/804 [=================

In [101]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled_pca,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.8329 - accuracy: 0.5297 - 553ms/epoch - 2ms/step
Loss: 0.8328554034233093, Accuracy: 0.5296792984008789


In [102]:
# Export our model to HDF5 file
nn3.save("AlphabetSoupCharity_Optimization.h5")

C:\Users\mikec\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
